## To do
- Fix image ID extraction.
  - Currently, img-ids are read from a directory listing.
  - Rather, use the img-ids from the CSV's dataframe - it's more fool-proof and avoids spurious subset errors.

- Run DenseNets and EfficientNets.
  - Ensure that you are saving the models at regular intervals - Colab disconnects randomly!

In [1]:
# pip install pandas


In [2]:
import os
import tensorflow as tf
import keras.utils
import numpy as np
import cv2
import pandas as pd

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)
import sys
sys.version

Num GPUs Available:  1
2.10.1


'3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]'

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf
tf.config.list_physical_devices('GPU')
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4672872508465917037
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5718933504
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14369175068350828410
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


#Load Data


In [5]:
df = pd.read_csv('ImageCLEFmedical_Caption_2023_concept_detection_train_labels.csv', sep="\t")

# Karthik
#df = pd.read_csv('/content/drive/MyDrive/Research/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/ImageCLEFmedical_Caption_2023_concept_detection_train_labels.csv', sep="\t")

In [6]:
df

,ID,cuis
0,ImageCLEFmedical_Caption_2023_train_000001,C0040405
1,ImageCLEFmedical_Caption_2023_train_000002,C0041618
2,ImageCLEFmedical_Caption_2023_train_000003,C0040405;C0817096;C0205271
3,ImageCLEFmedical_Caption_2023_train_000004,C0002978;C0036033;C0262950;C0225317
4,ImageCLEFmedical_Caption_2023_train_000005,C0040405;C0817096;C0497156
...,...,...
60913,ImageCLEFmedical_Caption_2023_train_060914,C0040405;C0005682
60914,ImageCLEFmedical_Caption_2023_train_060915,C1306645;C0000726;C1999039;C0203102;C0022646;C...
60915,ImageCLEFmedical_Caption_2023_train_060916,C0002978;C1510412;C0226333
60916,ImageCLEFmedical_Caption_2023_train_060917,C0002978;C1510412


#Find number of Classes

In [7]:
all_cui_train=df.loc[:,"cuis"]

In [8]:

sep_cui_train=[]
for cui_row in all_cui_train:
  #print(type(cui_row))
  sep_cui_train.extend(cui_row.split(sep=";"))
  
  
print(len(sep_cui_train))
unique_cui_train=np.array(sep_cui_train)
print(len(np.unique(unique_cui_train)))

227156
2125


In [9]:
df_valid = pd.read_csv('ImageCLEFmedical_Caption_2023_concept_detection_valid_labels.csv', sep="\t")

# Karthik
#df_valid = pd.read_csv('/content/drive/MyDrive/Research/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/ImageCLEFmedical_Caption_2023_concept_detection_valid_labels.csv', sep="\t")

In [10]:
df_valid.head()

,ID,cuis
0,ImageCLEFmedical_Caption_2023_valid_000001,C1306645;C0817096;C1996865;C0442800;C0018787;C...
1,ImageCLEFmedical_Caption_2023_valid_000002,C1306645;C0030797;C0016169;C0227668;C0087086
2,ImageCLEFmedical_Caption_2023_valid_000003,C1306645;C0006141;C0027651;C0332482;C0206692;C...
3,ImageCLEFmedical_Caption_2023_valid_000004,C0040405;C1306460
4,ImageCLEFmedical_Caption_2023_valid_000005,C0040405;C0035359;C0027651


In [11]:
df.loc[1875]

ID      ImageCLEFmedical_Caption_2023_train_001876
cuis           C1306645;C0817096;C0205129;C0039985
Name: 1875, dtype: object

In [12]:
all_cui_valid=df_valid.loc[:,"cuis"]

In [13]:
sep_cui_valid=[]
for cui_row in all_cui_valid:
  #print(type(cui_row))
  sep_cui_valid.extend(cui_row.split(sep=";"))
  
  
print(len(sep_cui_valid))
unique_cui_valid=np.array(sep_cui_valid)
print(len(np.unique(unique_cui_valid)))

40042
1945


In [14]:
train_and_valid_cuis=[]
train_and_valid_cuis.extend(unique_cui_train)
train_and_valid_cuis.extend(unique_cui_valid)

In [15]:
total_cui=np.array(train_and_valid_cuis)
print(len(np.unique(total_cui)))

2125


2125 is the total number of unique classes

#Label Encoding

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
encoder=LabelEncoder()
encoder.fit(total_cui)

LabelEncoder()

In [18]:
print(len(encoder.classes_))

2125


#Input Sequencer

In [19]:
df_train_copy=df

In [20]:
img_cui_dict=df_train_copy.set_index('ID').T.to_dict('list')

In [21]:
#print(img_cui_dict['ImageCLEFmedical_Caption_2023_train_001866'])

In [22]:
image_path='/Data/ImageCLEFmedical_Caption_2023_train_images/train'

# Karthik
#image_path = '/content/drive/MyDrive/Research/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/train'

In [23]:
#!ls /content/drive/MyDrive/ImageCLEFmedical-23/Captioning-Dataset_CLEF-2023/train | grep ".*\.jpg" | wc -l

#saving ids

In [24]:
#no need to run already saved
# img_id=[]
# for image in os.listdir(image_path):
#   img_id.append(image)

In [25]:
#no need to run already saved
# file_path_img_id = os.path.join(base_path,'img_id.npy')
# np.save(file_path_img_id, np.array(img_id))
# print("Saved", file_path_img_id)

In [26]:
img_id=[]
for id in df['ID']:
  id=id+'.jpg'
  img_id.append(id)
print(len(img_id))

60918


#Load Ids from file

In [27]:
#img_id = np.load(os.path.join(base_path, 'img_id.npy'))

In [28]:
print(len(img_id))
print(img_id[10])

60918
ImageCLEFmedical_Caption_2023_train_000011.jpg


In [29]:
print(img_cui_dict['ImageCLEFmedical_Caption_2023_train_039154.jpg'[:-4]][0].split(sep=";"))

['C1306645', 'C0817096', 'C1996865']


In [30]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
enc_labels=[]
for i in range(0,2125):
  enc_labels.append(i)
print(enc_labels)
mlb.fit([enc_labels])
print("Transformed:", mlb.transform([(661,2124,1),(11, 22)]))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [31]:
enc_tu=tuple([661])
print(enc_tu)
dd=[]
dd.append(enc_tu)
print(dd)

(661,)
[(661,)]


In [32]:
base_path = os.getcwd()

In [33]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, encoder=None, mlb=None, batch_size=8, image_size=224):
        self.ids = ids
        self.batch_size = batch_size
        self.image_size = image_size
        self.trainpath = base_path + '/Data/ImageCLEFmedical_Caption_2023_train_images/train'
        
        if encoder is not None:
          self.encoder = encoder
        if mlb is not None:
          self.mlb = mlb

        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path = base_path + '/Data/ImageCLEFmedical_Caption_2023_train_images/train'+'/'+id_name             
        
        ## Reading Image        
        image = cv2.imread(image_path, 1)
        cui_ids = img_cui_dict[id_name[:-4]][0].split(sep=";")        
        
        if(image is None):
          print("Error: ",id_name)
        try:
          image = cv2.resize(image, (self.image_size, self.image_size)).astype('float32')
          #print(image.shape)
        except:
          print("Skipping image: ")
        # Convert to float
        image /= 255
        return image, cui_ids
    
    def __getitem__(self, index):#For returning batch of images and thier cui ids
        # if(index+1)*self.batch_size > len(self.ids):
        #     self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        #print(len(files_batch))
        if(len(files_batch)<self.batch_size):
          files_batch.extend(self.ids[0:self.batch_size-len(files_batch)])
        
        images = []
        batch_cuis  = []
        for i,id_name in enumerate(files_batch):

          # while(not id_name.split('.')[1]=='jpg'): #to check if its not a jpg image
          #   id_name=files_batch[i+1]
          #   i+=1
          #   if(i>=self.batch_size):
          #     i=0
          
          _img, _cui_temp = self.__load__(id_name)
          images.append(_img)          
          batch_cuis.append(self.encoder.transform(_cui_temp))           
            
        images = np.array(images)
        encoded_batch_cuis = np.array(self.mlb.transform(batch_cuis))

        # print("Batch Encoded CUI IDs", encoded_batch_cuis)
        return images, encoded_batch_cuis
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [34]:
test_gen = DataGen(img_id, encoder, mlb)

In [35]:
test_gen.__getitem__(0)

(array([[[[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         [[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ]],
 
         ...,
 
         [[1.        , 1.        , 1.        ],
          [1.     

#Verifying Label Encoding

In [36]:
print(img_cui_dict['ImageCLEFmedical_Caption_2023_train_052471.jpg'[:-4]][0].split(sep=";"))

['C0040405']


In [37]:
encoder.transform(['C0040405'])

array([661])

In [38]:
mlb.transform([(661,)])[0][661]

1

Start with effecient net B0, in case of multiple GPU's Resnet 101 DenseNet 121

# Model Training

In [39]:
data_reader = DataGen(img_id, encoder, mlb, batch_size=1)
img,labels = data_reader[5]
print(img.shape)
print(labels)

(1, 224, 224, 3)
[[0 0 0 ... 0 0 0]]


In [40]:
print(keras.__version__)
print(tf.__version__)

2.10.0
2.10.1


In [41]:
# pip install EfficientNet

In [42]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Flatten, Input
# from keras import backend as K
# #from efficientnet.efficientnet.model import EfficientNetB0
# import efficientnet.keras

# from efficientnet.keras import EfficientNetB0
# IMG_SIZE=(224,224)

# #model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
# model = EfficientNetB0(
#         include_top=False,
#         #weights='imagenet',
#         input_shape=(*IMG_SIZE, 3)
#        )
# #model.summary(line_length=150)

# flatten = Flatten()
# new_layer2 = Dense(2125, activation='softmax', name='my_dense_2')

# inp2 = model.input
# out2 = new_layer2(flatten(model.output))

# opt = keras.optimizers.Adam(learning_rate=1e-05)


# model2 = Model(inp2, out2)
# model2.summary()
# model2.compile(
#     optimizer=opt,
#     loss='categorical_crossentropy',
#     metrics=['acc']
# )


# weight_save = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/ImageCLEFmedical-23/weights/weights-efficientnetb0/weights-epoch-{epoch:03d}.h5', save_weights_only=True, period=1)
# on_epoch_end_call = keras.callbacks.LambdaCallback(on_epoch_end=data_reader.on_epoch_end())

# #model2.load_weights('/home/miruna/LifeCLEF/FungiCLEF/weights/weights-efficientnetb0/weights-epoch-006.h5')
# model2.fit(data_reader,
#     epochs=10,
#     verbose=1,
#     steps_per_epoch=10,
#     callbacks=[weight_save, on_epoch_end_call]
# )

In [43]:
# tf.keras.applications.densenet.DenseNet121(
#     include_top=True,
#     weights='imagenet',
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation='softmax'
# )

In [44]:
# from sklearn.preprocessing import MultiLabelBinarizer

In [45]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Flatten, Input
# from keras import backend as K

# IMG_SIZE=(224,224)

In [46]:
# # model.summary(line_length=150)
# from tensorflow.keras.applications.densenet import DenseNet121




# model = DenseNet121(
#         include_top=False,
#         input_shape=(*IMG_SIZE, 3)
#        )
# flatten = Flatten()
# new_layer2 = Dense(2125, activation='softmax', name='my_dense_2')

# inp2 = model.input
# out2 = new_layer2(flatten(model.output))

# opt = keras.optimizers.Adam(learning_rate=1e-05)
# data_reader = DataGen(img_id)
# model2 = Model(inp2, out2)
# model2.summary()
# model2.compile(
#     optimizer=opt,
#     loss='sparse_categorical_crossentropy',
#     metrics=['acc']
# )


# #weight_save = keras.callbacks.ModelCheckpoint('weights/weights-efficientnetb0/weights-epoch-{epoch:03d}.h5', save_weights_only=True, period=1)
# #on_epoch_end_call = keras.callbacks.LambdaCallback(on_epoch_end=data_reader.on_epoch_end())

# #model2.load_weights('/home/miruna/LifeCLEF/FungiCLEF/weights/weights-efficientnetb0/weights-epoch-006.h5')
# model2.fit(data_reader,
#     epochs=10,
#     verbose=1,
#     steps_per_epoch=31000,
#     #callbacks=[weight_save, on_epoch_end_call]
# )

Resnet

In [47]:
# import keras
# from tensorflow.keras.applications.resnet import ResNet101
# from tensorflow.keras.applications.resnet import preprocess_input

# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Flatten, Input, Concatenate
# from keras import backend as K
# #ResNeXt101, preprocess_input = Classifiers.get('resnext101')
# IMG_SIZE=(224,224)

# model = ResNet101(
#         include_top=False,
#         weights='imagenet',
#         input_shape=(*IMG_SIZE, 3)
#        )

# flatten = Flatten()
# new_layer2 = Dense(2125, activation='softmax', name='my_dense_2')

# inp2 = model.input
# out2 = new_layer2(flatten(model.output))

# opt = keras.optimizers.Adam(learning_rate=1e-05)


# model2 = Model(inp2, out2)
# model2.summary()
# model2.compile(
#     optimizer=opt,
#     loss='categorical_crossentropy',
#     metrics=['acc']
# )


# weight_save = keras.callbacks.ModelCheckpoint(base_path+'/Data/resnetWeights/weights-epoch-{epoch:03d}.h5', save_weights_only=True, period=1)
# on_epoch_end_call = keras.callbacks.LambdaCallback(on_epoch_end=data_reader.on_epoch_end())

# model2.load_weights(base_path+'/Data/resnetWeights/weights-epoch-001.h5')
# model2.fit(data_reader,
#     epochs=10,
#     verbose=1,
#     steps_per_epoch=7625,
#     callbacks=[weight_save, on_epoch_end_call]
# )

effnet b2

In [48]:
# import sys, re, logging

# class Filter(object):
#     def __init__(self, stream):
#         self.stream = stream

#     def __getattr__(self, attr_name):
#         return getattr(self.stream, attr_name)

#     def write(self, data):
#         if not data.startswith("Executing op"):
#             self.stream.write(data)
#             self.stream.flush()

#     def flush(self):
#         self.stream.flush()
  
# sys.stdout = Filter(sys.stdout)
# sys.stderr = Filter(sys.stderr)

In [49]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from keras import backend as K
#from efficientnet.efficientnet.model import EfficientNetB0
import efficientnet.keras

from efficientnet.keras import EfficientNetB2
from keras.callbacks import CSVLogger


IMG_SIZE=(224,224)

#model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
model = EfficientNetB2(
        include_top=False,
        #weights='imagenet',
        input_shape=(*IMG_SIZE, 3)
       )
#model.summary(line_length=150)

flatten = Flatten()
new_layer2 = Dense(2125, activation='softmax', name='my_dense_2')

inp2 = model.input
out2 = new_layer2(flatten(model.output))

opt = keras.optimizers.Adam(learning_rate=1e-05)

model2 = Model(inp2, out2)
model2.summary()
model2.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['acc']
)


weight_save = keras.callbacks.ModelCheckpoint(base_path+'/Data/efficientnetb2Weights/weights-epoch-{epoch:004d}.h5', save_weights_only=True, period=1)
on_epoch_end_call = keras.callbacks.LambdaCallback(on_epoch_end=data_reader.on_epoch_end())
csv_logger = CSVLogger('/Data/efficientnetb2Weights/training.log')



model2.load_weights(base_path+'/Data/efficientnetb2Weights/weights-epoch-001.h5')


# loss, acc = model.evaluate(test_images, test_labels, verbose=2)
# print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessTruncatedNormalV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _Eag

Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessTruncatedNormalV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill i

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StatelessTruncatedNormalV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill i

Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/rep

Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/rep

...

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

In [50]:
model2.fit(data_reader,
    epochs=20,
    verbose=2,
    steps_per_epoch=7625,
    callbacks=[weight_save, on_epoch_end_call, csv_logger]
)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp i

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:l

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:l

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:l

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:l

...

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:loc

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /j

Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:loc

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /j

NotFoundError: Failed to create a NewWriteableFile: /Data/efficientnetb2Weights/training.log : The system cannot find the path specified.
; No such process